In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date

In [10]:
from pyspark.sql import SparkSession

# Criando a sessão do Spark conectando ao cluster
spark = SparkSession.builder \
    .master("spark://spark-container:7077") \
    .appName("Teste Cluster") \
    .getOrCreate()

# Criando um DataFrame de exemplo
data = [(1, "Alice", 25), (2, "Bob", 30), (3, "Charlie", 35)]
columns = ["id", "nome", "idade"]

df = spark.createDataFrame(data, columns)

# Mostrar os dados no console
df.show()

# Fazer um cálculo simples (média de idade)
df.selectExpr("AVG(idade) as idade_media").show()

# Parar a sessão do Spark
spark.stop()


+---+-------+-----+
| id|   nome|idade|
+---+-------+-----+
|  1|  Alice|   25|
|  2|    Bob|   30|
|  3|Charlie|   35|
+---+-------+-----+

+-----------+
|idade_media|
+-----------+
|       30.0|
+-----------+



In [7]:
def ingest(spark, projeto, table, format, schema=None):
    raw = f"s3a://raw/{projeto}/{table}/*"
    bronze = f"s3a://data/bronze/{projeto}/{table}"

    df = spark.read.format(format).option('header','true').schema(schema).load(raw)
    df.write.mode('overwrite').save(bronze)

In [8]:
arqschema = "Data STRING, Valor DOUBLE, Identificador STRING, Descricao STRING"
# arqschema = "date DATE, title STRING, amount DOUBLE"
ingest(spark,'nubank','extrato','csv',arqschema)

Py4JJavaError: An error occurred while calling o50.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Da